# PRACTICA 1
### Rubén Gómez Márquez

In [19]:
#En primer lugar importamos las librerias que vamos a utilizar. 
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs

#Vamos a extraer y hacer un análisis de los diferentes ordenadores portátiles por sus precios y marcas de la página de
#mediamark. La finalidad es hacer un estudio de mercado de un producto en concreto, que puede ser por ejemplo un ordenador.

#La página en cuestión es la siguiente: 
#https://www.mediamarkt.es/es/category/_port%C3%A1tiles-701175.html?searchParams=&sort=&view=PRODUCTGRID&page=1
#Observamos que tiene varios apartados, por lo que tendremos que extraer diferentes url y analizarlos todos.

base_url='https://www.mediamarkt.es/es/category/_port%C3%A1tiles-701175.html?searchParams=&sort=&view=PRODUCTGRID&page='


#CALCULO DEL NÚMERO DE PÁGINAS

#En esta parte lo que haremos será generar una lista que incluya el numero de páginas de esta forma:
#['1','2','3','4','5',...]. Para ello nos introduciremos en la página y buscaremos si la página tiene
#un tag asociado a 'Continuar', si lo tiene añadirá el número de página a la lista y pasará a la siguiente.
#Cuando no encuentre el tag, añadirá el número de página y rompera el bucle.

#Es necesario acceder a una página de inicio, que será la primera y luego el url se irá modificando.
url='https://www.mediamarkt.es/es/category/_port%C3%A1tiles-701175.html?searchParams=&sort=&view=PRODUCTGRID&page=1'
page=requests.get(url)
soup=bs(page.content,"html.parser")
count=1
pages=list()
soup.find('li',{'class':'pagination-next'})

#Hacemos un bucle while con un número exagerado de iteraciones, para asegurarnos que abarcamos
#todas las posibles páginas y sabiendo que se romperá.
while count<1000:
    #Esta es la condición, el continuar se encuentra en el tag li de clase pagination-next
    if soup.find('li',{'class':'pagination-next'}) is not None:
        #Aquí añadimos el número de la página a la lista que es precisamente el contador
        pages.append(str(count))
        #Sumamos uno al contador y modificamos el url utilizando la base que no tiene asociada
        #ningún número de página
        count=count+1
        url=base_url+str(count)
        #Nos introducimos en la página nueva y cambiamos el soup, que modificara la condición a su vez.
        page=requests.get(url)
        soup=bs(page.content,"html.parser")
    else:
        pages.append(str(count))
        break

#OBTENIENDO LOS ELEMENTOS DE INTERES

        
#Aqui definimos los vectores que van a acumular los datos que luego añadiremos
#al dataframe. Es importante que esten fuera de los bucles, porque si no se 
#resetean y no almacenan los resultados de una página a otra.
marcas=list()
valoraciones=list()
nombre=list()
precio_actual=list()
precio_anterior=list()
ahorro=list()

#Este primer bucle avanza a lo largo de las páginas
for i in pages:
    #Url de cada página
    url=base_url + i
      
    #Hacemos el request de la página
    page=requests.get(url)
    soup=bs(page.content,"html.parser")
    
    #MARCA Y VALORACION
    #Accedemos al tag que contiene la marca y la valoración
    base_content=soup.findAll('div',{'class':'base-content'})
    #Cada producto tiene uno asique podemos recorrer todos los productos en bucle
    for i in base_content:
        
        #MARCA
        marca=i.find('a',{'class':'manufacturer'})
        marcas.append(marca.img['alt'])
        
        #VALORACION
        #Para la valoración hay que recurrir a un bucle condicional ya que algunos
        #productos no están valorados y por lo tanto dentro del tag div de clase 'rate'
        #no hay ningún tag 'div'.
        valoracion=i.find('div',{'class':'rate'})
        if valoracion.div is None:
            valoraciones.append('No Valorado')
        else:
            valoraciones.append(valoracion.div.div['class'])
            
    #NOMBRE Y PRECIO
    #De forma similar obtenemos los diferentes precios y el nombre del producto.
    base_info=soup.findAll('div',{'class':"base-info"})
    for i in base_info:
        
        #NOMBRE
        nombre.append(i.h3.a.text)
        
        #PRECIO
        wrapper=i.find('div',{'class':'info-wrapper'})
        #Esta condición baraja aquellos productos que se encuentran agotados.
        if wrapper.div is None:
            precio_actual.append('Agotado')
            precio_anterior.append('Agotado')
            ahorro.append('Agotado')
        else:
            precios=i.findAll('div',{'class':'price'})
            #Esta condición baraja aquellos productos que no han sufrido un descuento.
            if len(precios)>1:
                precio_actual.append(precios[1].text)
                precio_anterior.append(precios[0].text)
                ahorros=i.find('div',{'class':"price-old-you-save"})
                ahorro.append(ahorros.text)
            else:
                precio_actual.append(precios[0].text)
                precio_anterior.append('Sin descuento')
                ahorro.append('Sin descuento')
    
#El dataframe obtenido podemos observar que debe ser limpiado si se quieren analizar los datos que se obtienen,
#pero en el se encuentran todos los datos que nos interesan.Incluso más datos dentro del propio nombre que 
#indican características del propio producto.
df=pd.DataFrame({"Marca":marcas,"Nombre":nombre,"Precio_Actual":precio_actual,"Precio_Antiguo":precio_anterior,"Descuento":ahorro,"Valoracion":valoraciones},index=list(range(1,len(nombre)+1)))   
df.to_csv('Portatiles.csv',index=False, sep=";")


In [20]:
#AQUI PODEMOS OBSERVAR LOS RESULTADOS EN EL DATAFRAME, QUE SE REGISTRARAN EN EL ARCHIVO CSV.
df

,Marca,Nombre,Precio_Actual,Precio_Antiguo,Descuento,Valoracion
1,ACER,"\r\n\t\t\tPortátil - Acer Aspire 3 A315-54K, 1...","499,-","\n\n\r\n\t\t550,-","\r\n\t\tTe ahorras: € 51,00",No Valorado
2,HP,"\r\n\t\t\tPortátil - HP 15s-fq1008ns, 15.6"" HD...","599,-","\n\n\r\n\t\t655,-","\r\n\t\tTe ahorras: € 56,00",[value-3-5]
3,LENOVO,"\r\n\t\t\tPortátil - Lenovo S145-15IIL, 15.6 ""...","599,-",Sin descuento,Sin descuento,No Valorado
4,HP,"\r\n\t\t\tPortátil - HP 15-db1001ns, 15.6"" HD,...","399,-","\n\n\r\n\t\t441,-","\r\n\t\tTe ahorras: € 42,00",[value-5]
5,ACER,\r\n\t\t\tPortátil - Acer Aspire 3 A315-54K-30...,"399,-","\n\n\r\n\t\t439,-","\r\n\t\tTe ahorras: € 40,00",No Valorado
...,...,...,...,...,...,...
233,ASUS,\r\n\t\t\tPortátil gaming - ASUS ROG Zephyrus ...,"3499,-","\n\n\r\n\t\t3789,-","\r\n\t\tTe ahorras: € 290,00",No Valorado
234,TOSHIBA,\r\n\t\t\tPortátil - Toshiba Portege Z30-C-10W...,"1322,-",Sin descuento,Sin descuento,No Valorado
235,LENOVO,\r\n\t\t\tREACONDICIONADO Portátil - Lenovo Id...,"983,20","\n\n\r\n\t\t1229,-","\r\n\t\tTe ahorras: € 245,80",No Valorado
236,TOSHIBA,"\r\n\t\t\tConvertible 2 en 1 - TOSHIBA, Portég...","1106,-",Sin descuento,Sin descuento,No Valorado
